In [49]:
import pandas as pd
import re

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 5)
pd.set_option('display.width', 1280)

if __name__ == "__main__":
    df = pd.read_csv('Data Processing Specialist Dataset.csv')
    
    print(df[['Name ', 'Allergens ']])

                                          Name                     Allergens 
0                              Noël, Grimbergen               Gluten, barley.
1                Blågårds Pilsner 5.1% organic.                        Gluten
2                          Blanche 4.5%, Harboe                        Gluten
3     Golden Naked Christmas Ale 7.5%, Jacobsen  Gluten, barley, oats, wheat.
4              Donker Winter Ale 8.9%, Jacobsen        Gluten, barley, wheat.
...                                         ...                           ...
1994       Vegetable Fund Fond "De Chef", Knorr                        Celery
1995                     Vegetable broth, Knorr                              
1996                    Bouillon chicken, Knorr                              
1997                          Beef broth, Knorr                              
1998                          Pork broth, Knorr                       Celery.

[1999 rows x 2 columns]


In [50]:
    # Allergene normalisieren und als Liste extrahieren
    def extract_allergens(entry):
        if pd.isna(entry) or entry.strip() == "":
            return []
        entry = entry.lower()
        entry = re.sub(r"[.]", "", entry)
        entry = "" if entry == "-" else entry
        parts = re.split(r",", entry)
        return [p.strip() for p in parts if p.strip()]
    
    df['Allergen_List'] = df['Allergens '].apply(extract_allergens)

    print(df[['Name ', 'Allergens ', 'Allergen_List']])

                                          Name                     Allergens                   Allergen_List
0                              Noël, Grimbergen               Gluten, barley.               [gluten, barley]
1                Blågårds Pilsner 5.1% organic.                        Gluten                       [gluten]
2                          Blanche 4.5%, Harboe                        Gluten                       [gluten]
3     Golden Naked Christmas Ale 7.5%, Jacobsen  Gluten, barley, oats, wheat.  [gluten, barley, oats, wheat]
4              Donker Winter Ale 8.9%, Jacobsen        Gluten, barley, wheat.        [gluten, barley, wheat]
...                                         ...                           ...                            ...
1994       Vegetable Fund Fond "De Chef", Knorr                        Celery                       [celery]
1995                     Vegetable broth, Knorr                                                           []
1996               

In [51]:
    # Tabelle "aufblasen", sodass jede Zeile genau 1 Allergen enthält
    exploded = df.explode('Allergen_List')

    print(exploded[['Name ', 'Allergens ', 'Allergen_List']])

                                          Name                     Allergens  Allergen_List
0                              Noël, Grimbergen               Gluten, barley.        gluten
0                              Noël, Grimbergen               Gluten, barley.        barley
1                Blågårds Pilsner 5.1% organic.                        Gluten        gluten
2                          Blanche 4.5%, Harboe                        Gluten        gluten
3     Golden Naked Christmas Ale 7.5%, Jacobsen  Gluten, barley, oats, wheat.        gluten
...                                         ...                           ...           ...
1994       Vegetable Fund Fond "De Chef", Knorr                        Celery        celery
1995                     Vegetable broth, Knorr                                         NaN
1996                    Bouillon chicken, Knorr                                         NaN
1997                          Beef broth, Knorr                                 

In [52]:
    # Gruppierung nach Allergenen
    allergen_groups = exploded.groupby('Allergen_List').agg({
        'Name ': lambda x: list(pd.Series(x).dropna().unique()),  # Liste der Produktnamen
    })

    print(allergen_groups)

                                                                                Name 
Allergen_List                                                                        
all nuts                            [Gift pack with chocolate, Perto, Gift pack wi...
allergens be-kind bar: peanuts          [Snickers + Be-Kind bar with sea salt & nuts]
allergens snickers creamy: peanuts  [Snickers Creamy + M&M's Peanuts + Skittles Fr...
almond                              [Almond eggs, Anthon Berg, Marzipan eggs with ...
almonds                             [Cardamom snails mini, Pågen, Roulade with mar...
...                                                                               ...
wheat (gluten-containing grain)     [Ich Bin Blueberry Berliner Weisse 3.7%, Mikke...
wheat and gluten-containing grains  [Porridge balls, Kohberg, Protein balls, Kohbe...
wheat and its derivatives                     [Brooklyn Pulp Art Hazy IPA 6.0% (can)]
wheat flour                             [Dessert panca

In [53]:
    # Gesamtzahl Produkte (für Prozentangabe)
    total_products = df['Name '].count()

    print("total products: ", total_products)

total products:  1999


In [54]:
    # Spalte für Anzahl Produkte
    allergen_groups['Count'] = allergen_groups['Name '].apply(len)

    print(allergen_groups)

                                                                                Name   Count
Allergen_List                                                                               
all nuts                            [Gift pack with chocolate, Perto, Gift pack wi...      4
allergens be-kind bar: peanuts          [Snickers + Be-Kind bar with sea salt & nuts]      1
allergens snickers creamy: peanuts  [Snickers Creamy + M&M's Peanuts + Skittles Fr...      1
almond                              [Almond eggs, Anthon Berg, Marzipan eggs with ...      3
almonds                             [Cardamom snails mini, Pågen, Roulade with mar...     82
...                                                                               ...    ...
wheat (gluten-containing grain)     [Ich Bin Blueberry Berliner Weisse 3.7%, Mikke...      2
wheat and gluten-containing grains  [Porridge balls, Kohberg, Protein balls, Kohbe...      9
wheat and its derivatives                     [Brooklyn Pulp Art Hazy 

In [55]:
    # Prozent-Anteil berechnen
    allergen_groups['% Share'] = (allergen_groups['Count'] / total_products * 100).round(1)

    print(allergen_groups)

                                                                                Name   Count  % Share
Allergen_List                                                                                        
all nuts                            [Gift pack with chocolate, Perto, Gift pack wi...      4      0.2
allergens be-kind bar: peanuts          [Snickers + Be-Kind bar with sea salt & nuts]      1      0.1
allergens snickers creamy: peanuts  [Snickers Creamy + M&M's Peanuts + Skittles Fr...      1      0.1
almond                              [Almond eggs, Anthon Berg, Marzipan eggs with ...      3      0.2
almonds                             [Cardamom snails mini, Pågen, Roulade with mar...     82      4.1
...                                                                               ...    ...      ...
wheat (gluten-containing grain)     [Ich Bin Blueberry Berliner Weisse 3.7%, Mikke...      2      0.1
wheat and gluten-containing grains  [Porridge balls, Kohberg, Protein balls, Kohbe

In [56]:
    # Index umbenennen und zurücksetzen
    allergen_groups = allergen_groups.reset_index().rename(columns={'Allergen_List': 'Allergen', 'Name ': 'Products'}).sort_values(by='Count', ascending=False)

    print(allergen_groups)

                          Allergen                                           Products  Count  % Share
30                          gluten  [Noël, Grimbergen, Blågårds Pilsner 5.1% organ...    562     28.1
45                            milk  [Bailey's 17%, Llama puffs with cheese & onion...    424     21.2
67                             soy  [Subsidized leafing from 6 months. Eco. Arla b...    230     11.5
80                           wheat  [Golden Naked Christmas Ale 7.5%, Jacobsen, Do...    147      7.4
25                            eggs  [Tea bun, Carrot bun with sunflower seeds, ⭐️ ...    131      6.6
..                             ...                                                ...    ...      ...
72                         spelled  [Elderflower Session IPA 5.0% organic. Nørrebr...      1      0.1
75                        sulphite                                  [Bia Saigon 4.9%]      1      0.1
70  soya\nallergens snickers: milk      [Snickers + Be-Kind bar with sea salt & nu